In [2]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
from tabulate import tabulate

In [3]:
def readData(file):
	"""
	reads the ground truth file
	returns a 2D array with each
	row as GT pose(arranged row major form)
	array size should be 1101*12
	"""
	data = []
	for i in range(77):
		line = file.readline()
		line = line.split()
		arr = np.array(line)
		arr = arr.reshape(3, 4)
		data.append(arr)
	return data

In [4]:
def readPointCloud(filename):
	"""
	reads bin file and returns
	as m*4 np array
	all points are in meters
	you can filter out points beyond(in x y plane)
	50m for ease of computation
	and above or below 10m
	"""
	pcl = np.fromfile(filename, dtype=np.float32,count=-1)
	pcl = pcl.reshape([-1,4])
	return pcl

In [5]:
finalPointCloud=[]
pointsInWorldFrame=[]
txtfile=open("01.txt","r")
transforms = readData(txtfile)
txtfile.close()   

In [6]:

def reconstructMat(alpha,beta,gamma):
   
    Rz = np.array([[np.cos(alpha),-1*np.sin(alpha),0],
                   [np.sin(alpha),np.cos(alpha),0],
                   [0,0,1]])
    Ry = np.array([[np.cos(beta),0,np.sin(beta)],
                   [0,1,0],
                   [-1*np.sin(beta),0,np.cos(beta)]])
    Rx = np.array([[1,0,0],
                   [0,np.cos(gamma),-1*np.sin(gamma)],
                   [0,np.sin(gamma),np.cos(gamma)]])
    R = np.matmul(np.matmul(Rz,Ry),Rx)
    return R

In [7]:
loc = "./01/0000" + str(10) + ".bin"
ar = readPointCloud(loc)
arr = np.array(ar[:,:3])
color = np.array(arr[:,:-1])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(arr)
tr = np.append(transforms[10],[[0,0,0,1]],axis=0)
o3d.geometry.PointCloud.transform(pcd,tr)
pcdf = pcd
for i in range(11,87):
    loc = "./01/0000" + str(i) + ".bin"
    ar = readPointCloud(loc)
    arr = np.array(ar[:,:3])
    color = np.array(arr[:,:-1])
    R = reconstructMat(0,-np.pi/2,np.pi/2)
    t = np.matmul(arr,R).astype(dtype = 'float')
    tr = np.array(transforms[i-11]).astype(dtype = 'float')
    ans = np.matmul(t,tr) 
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointsInWorldFrame)
    pcdf += pcd
    

In [8]:
o3d.visualization.draw_geometries([pcdf])
o3d.io.write_point_cloud("./pcd.ply",pcdf)

True